In [2]:
import os
from catboost import CatBoostRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import gc
gc.disable()

In [ ]:
# .mean() - среднее
# максимальная длина ego_id = 13 символов
# как проверить на сколько значим признак в sсikitlear

In [1]:
DATA_DIR = 'data'

### Преобразование датасета train

In [3]:
df_train = pd.read_csv(
    os.path.join(DATA_DIR, 'train.csv'),
    dtype='str',
    encoding='utf-8',
    # usecols=['ego_id', 'u', 'v', 't', 'x2', 'x3'],
    # nrows=1000
)
df_train['ego_id'] = df_train['ego_id'].astype('str')
df_train['u'] = df_train['u'].astype('str')
df_train['v'] = df_train['v'].astype('str')
df_train['t'] = df_train['t'].astype('float').fillna(0.0)
print(len(df_train))
df_train.head(3)

122280372


,ego_id,u,v,t,x1,x2,x3
0,0,131,84,148.0,5.669199792045917e-07,0.0,0.0
1,0,135,164,396.7,0.0624627359881627,0.0,0.0
2,0,47,15,0.0,0.0,0.0,1.0


In [4]:
df_attr = pd.read_csv(
    os.path.join(DATA_DIR, 'attr.csv'),
    dtype='str',
    encoding='utf-8',
    # nrows=1000
)
print(len(df_attr))
df_attr.head(3)

14930748


,ego_id,u,age,city_id,sex,school,university
0,0,227,68,-1,1,778293348,-1
1,0,45,38,237065842,1,82803468,238500268
2,0,142,60,237065842,1,196560139,-1


In [5]:
df = pd.merge(df_train, df_attr, on=['ego_id', 'u'], how='left')
print(len(df))
df

122280372


,ego_id,u,v,t,x1,x2,x3,age,city_id,sex,school,university
0,0,131,84,148.0,5.669199792045917e-07,0.0,0.0,62,237065842,1,-1,-1
1,0,135,164,396.7,0.0624627359881627,0.0,0.0,45,238321946,2,461075276,991369526
2,0,47,15,0.0,0.0,0.0,1.0,41,-1,2,-1,-1
3,0,5,4,594.5,0.049629738619421,0.0,0.0,49,237065842,1,929914814,900704564
4,0,176,219,45.5,1.2379350629578283,0.0,0.0,21,237065842,1,967669435,149014508
...,...,...,...,...,...,...,...,...,...,...,...,...
122280367,1709396984692,3,5,34.6,2.307749740790665,1.0986122886681098,0.0,16,-1,2,-1,-1
122280368,1709396984692,1,5,53.8,3.729142613395535,3.49650756146648,1.0,23,609282824,2,-1,-1
122280369,1709396984692,1,7,1.5,4.2869843553809766,0.0,0.0,23,609282824,2,-1,-1
122280370,1709396984692,5,11,2.0,3.500756590674839,0.0,0.0,17,-1,2,923370753,-1


In [6]:
del df_train, df_attr

In [7]:
df['node_1'] = df['ego_id'] + '_' + df['u']
df['node_2'] = df['ego_id'] + '_' + df['v']

In [8]:
df['node_1'] = df['node_1'].astype('category').cat.codes
df['node_2'] = df['node_2'].astype('category').cat.codes

In [9]:
df

,ego_id,u,v,t,x1,x2,x3,age,city_id,sex,school,university,node_1,node_2
0,0,131,84,148.0,5.669199792045917e-07,0.0,0.0,62,237065842,1,-1,-1,36,261
1,0,135,164,396.7,0.0624627359881627,0.0,0.0,45,238321946,2,461075276,991369526,40,68
2,0,47,15,0.0,0.0,0.0,1.0,41,-1,2,-1,-1,218,53
3,0,5,4,594.5,0.049629738619421,0.0,0.0,49,237065842,1,929914814,900704564,221,217
4,0,176,219,45.5,1.2379350629578283,0.0,0.0,21,237065842,1,967669435,149014508,83,125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122280367,1709396984692,3,5,34.6,2.307749740790665,1.0986122886681098,0.0,16,-1,2,-1,-1,5212705,5544904
122280368,1709396984692,1,5,53.8,3.729142613395535,3.49650756146648,1.0,23,609282824,2,-1,-1,5212696,5544904
122280369,1709396984692,1,7,1.5,4.2869843553809766,0.0,0.0,23,609282824,2,-1,-1,5212696,5544906
122280370,1709396984692,5,11,2.0,3.500756590674839,0.0,0.0,17,-1,2,923370753,-1,5212707,5544895


In [11]:
df.to_csv(
    os.path.join(DATA_DIR, 'df_train_attr.csv'),
    index=False,
    encoding='utf-8'
)

In [10]:
df.to_pickle(
    os.path.join(DATA_DIR, 'df_train_attr.pkl')
)

In [13]:
del df

### Catboost

In [4]:
df = pd.read_pickle(
    os.path.join(DATA_DIR, 'df_train_attr.pkl')
)
df

,ego_id,u,v,t,x1,x2,x3,age,city_id,sex,school,university,node_1,node_2
0,0,131,84,148.0,5.669199792045917e-07,0.0,0.0,62,237065842,1,-1,-1,36,261
1,0,135,164,396.7,0.0624627359881627,0.0,0.0,45,238321946,2,461075276,991369526,40,68
2,0,47,15,0.0,0.0,0.0,1.0,41,-1,2,-1,-1,218,53
3,0,5,4,594.5,0.049629738619421,0.0,0.0,49,237065842,1,929914814,900704564,221,217
4,0,176,219,45.5,1.2379350629578283,0.0,0.0,21,237065842,1,967669435,149014508,83,125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122280367,1709396984692,3,5,34.6,2.307749740790665,1.0986122886681098,0.0,16,-1,2,-1,-1,5212705,5544904
122280368,1709396984692,1,5,53.8,3.729142613395535,3.49650756146648,1.0,23,609282824,2,-1,-1,5212696,5544904
122280369,1709396984692,1,7,1.5,4.2869843553809766,0.0,0.0,23,609282824,2,-1,-1,5212696,5544906
122280370,1709396984692,5,11,2.0,3.500756590674839,0.0,0.0,17,-1,2,923370753,-1,5212707,5544895


In [5]:
train = df[df['ego_id'].isin(list(df['ego_id'].unique())[:10000])]
train = train.dropna(
    subset=['x2', 'x3', 'age', 'city_id', 'sex', 'school', 'university'],
    axis=0,
    how='any'
)
# train.to_pickle('data/saved_train.pkl')
print(len(train))
train.head(3)

18532216


,ego_id,u,v,t,x1,x2,x3,age,city_id,sex,school,university,node_1,node_2
0,0,131,84,148.0,5.669199792045917e-07,0.0,0.0,62,237065842,1,-1,-1,36,261
1,0,135,164,396.7,0.0624627359881627,0.0,0.0,45,238321946,2,461075276,991369526,40,68
2,0,47,15,0.0,0.0,0.0,1.0,41,-1,2,-1,-1,218,53


In [6]:
test = df[df['ego_id'].isin(list(df['ego_id'].unique())[10000:12000])]
test = test.dropna(
    subset=['x2', 'x3', 'age', 'city_id', 'sex', 'school', 'university'],
    axis=0,
    how='any'
)
# test.to_pickle('data/saved_test.pkl')
print(len(test))
test.head(3)

3780169


,ego_id,u,v,t,x1,x2,x3,age,city_id,sex,school,university,node_1,node_2
19918223,274877907171,0,44,115.1,0.1355923642278693,0.0,0.0,14,790811587,2,-1,-1,6054551,6439187
19918224,274877907171,17,11,21.5,0.0474604931367079,0.0,0.0,52,790811587,2,-1,-1,6054560,6439154
19918225,274877907171,5,6,594.5,2.1557209380380813e-07,0.0,0.0,34,790811587,2,625502603,-1,6054585,6439194


In [7]:
val = df[df['ego_id'].isin(list(df['ego_id'].unique())[12000:14000])]
val = val.dropna(
    subset=['x2', 'x3', 'age', 'city_id', 'sex', 'school', 'university'],
    axis=0,
    how='any'
)
# val.to_pickle('data/saved_val.pkl')
print(len(val))
val.head(3)

3715001


,ego_id,u,v,t,x1,x2,x3,age,city_id,sex,school,university,node_1,node_2
23985430,326417515014,253,171,84.4,0.0002656269207876577,0.0,0.0,18,-1,2,-1,-1,6435518,6843404
23985431,326417515014,140,216,4.9,2.7537270345609026,2.8903717578961645,0.0,17,-1,1,-1,-1,6435404,6843450
23985432,326417515014,116,141,40.1,0.4712812585101567,0.0,0.0,17,572040791,1,169775667,-1,6435381,6843374


In [8]:
del df

In [12]:
trainn = pd.read_pickle('data/saved_train.pkl')
train = trainn[trainn['ego_id'].isin(list(trainn['ego_id'].unique())[:1000])]
print(len(train))
train.head(3)

1937384


,ego_id,u,v,t,x1,x2,x3,age,city_id,sex,school,university,node_1,node_2
0,0,131,84,148.0,5.669199792045917e-07,0.0,0.0,62,237065842,1,-1,-1,36,261
1,0,135,164,396.7,0.0624627359881627,0.0,0.0,45,238321946,2,461075276,991369526,40,68
2,0,47,15,0.0,0.0,0.0,1.0,41,-1,2,-1,-1,218,53


In [13]:
# test = pd.read_pickle('data/saved_test.pkl')
test = trainn[trainn['ego_id'].isin(list(trainn['ego_id'].unique())[1000:1200])]
print(len(test))
test.head(3)

341667


,ego_id,u,v,t,x1,x2,x3,age,city_id,sex,school,university,node_1,node_2
2087478,25769804021,7,14,228.7,0.0001928890294815015,0.0,0.0,32,870128347,2,297359996,656866418,5941306,6317850
2087479,25769804021,101,38,134.9,3.7869421960028085e-09,0.0,0.0,17,-1,2,-1,-1,5941161,6317913
2087480,25769804021,44,35,0.0,0.0,0.0,1.0,29,58668681,2,211809729,461988711,5941278,6317910


In [14]:
val = pd.read_pickle('data/saved_val.pkl')
val = trainn[trainn['ego_id'].isin(list(trainn['ego_id'].unique())[1200:1400])]
print(len(val))
val.head(3)

363435


,ego_id,u,v,t,x1,x2,x3,age,city_id,sex,school,university,node_1,node_2
2453274,25769804566,60,24,0.0,0.0,2.079441541679836,0.0,15,-1,1,-1,-1,5974043,6353621
2453275,25769804566,18,0,0.0,0.0,0.0,1.0,16,-1,1,-1,-1,5974000,6353606
2453276,25769804566,19,42,0.0,0.0,0.0,1.0,-1,-1,-1,-1,-1,5974001,6353640


In [15]:
# features
X = ['x2', 'x3', 'age', 'city_id', 'sex', 'school', 'university']
cat_features = ['x3', 'city_id', 'sex', 'school', 'university']
# target (целевой признак)
y = ['x1']

# x2         - числовой
# x3         - категориальный
# age        - числовой
# city       - категориальный
# sex        - категориальный
# school     - категориальный
# university - категориальный

In [16]:
model = CatBoostRegressor(
    cat_features=cat_features,
    eval_metric='RMSE',
    iterations=100,
    depth=16,
    random_seed=48,
    # bagging_temperature=0.2,
    od_type='Iter',
    task_type='GPU',
    devices='0:1'
)

In [17]:
model.fit(
    train[X], train[y],
    eval_set = (val[X], val[y])
)

Learning rate set to 0.271101
0:	learn: 1.1785085	test: 1.1859721	best: 1.1859721 (0)	total: 446ms	remaining: 44.2s
1:	learn: 1.0804408	test: 1.0778013	best: 1.0778013 (1)	total: 847ms	remaining: 41.5s
2:	learn: 1.0234911	test: 1.0140066	best: 1.0140066 (2)	total: 1.27s	remaining: 41.2s
3:	learn: 0.9911922	test: 0.9777867	best: 0.9777867 (3)	total: 1.68s	remaining: 40.4s
4:	learn: 0.9729991	test: 0.9572840	best: 0.9572840 (4)	total: 2.11s	remaining: 40.1s
5:	learn: 0.9628977	test: 0.9457078	best: 0.9457078 (5)	total: 2.52s	remaining: 39.5s
6:	learn: 0.9506980	test: 0.9401181	best: 0.9401181 (6)	total: 2.94s	remaining: 39s
7:	learn: 0.9435608	test: 0.9368236	best: 0.9368236 (7)	total: 3.35s	remaining: 38.5s
8:	learn: 0.9395343	test: 0.9349181	best: 0.9349181 (8)	total: 3.77s	remaining: 38.1s
9:	learn: 0.9348731	test: 0.9338904	best: 0.9338904 (9)	total: 4.18s	remaining: 37.7s
10:	learn: 0.9319929	test: 0.9333013	best: 0.9333013 (10)	total: 4.59s	remaining: 37.2s
11:	learn: 0.9300835	tes

In [19]:
test = pd.read_csv('data/test.csv', dtype='str', encoding='utf-8')
test

,ego_id,u,v,t,x1,x2,x3
0,8,20,19,185.7,0.0003839089038801514,0.0,0.0
1,8,131,125,161.4,0.4034464087119478,0.0,0.0
2,8,73,56,127.0,8.554643084493821e-05,0.0,0.0
3,8,0,4,594.5,0.2886418426336004,0.0,0.0
4,8,63,73,127.0,4.28169159968051e-07,0.0,0.0
...,...,...,...,...,...,...,...
40548775,1709396984676,89,0,4.8,1.1678430495988743,0.0,0.0
40548776,1709396984676,84,87,NaN,1.1790997584961869e-06,0.0,0.0
40548777,1709396984676,8,18,83.0,1.1751815432451471,0.0,0.0
40548778,1709396984676,33,20,28.3,0.5511019429181079,0.0,0.0


In [21]:
test.columns

Index(['ego_id', 'u', 'v', 't', 'x1', 'x2', 'x3'], dtype='object')

In [25]:
test = test[['ego_id', 'u', 'v', 't', 'x2', 'x3']]
test

,ego_id,u,v,t,x2,x3
0,8,20,19,185.7,0.0,0.0
1,8,131,125,161.4,0.0,0.0
2,8,73,56,127.0,0.0,0.0
3,8,0,4,594.5,0.0,0.0
4,8,63,73,127.0,0.0,0.0
...,...,...,...,...,...,...
40548775,1709396984676,89,0,4.8,0.0,0.0
40548776,1709396984676,84,87,NaN,0.0,0.0
40548777,1709396984676,8,18,83.0,0.0,0.0
40548778,1709396984676,33,20,28.3,0.0,0.0


In [24]:
attr = pd.read_csv('data/attr.csv', dtype='str', encoding='utf-8')
attr

,ego_id,u,age,city_id,sex,school,university
0,0,227,68,-1,1,778293348,-1
1,0,45,38,237065842,1,82803468,238500268
2,0,142,60,237065842,1,196560139,-1
3,0,280,66,-1,2,963209731,720783270
4,0,41,18,-1,2,308862409,-1
...,...,...,...,...,...,...,...
14930743,1709396984692,2,16,492149712,2,769209871,-1
14930744,1709396984692,12,15,-1,1,-1,-1
14930745,1709396984692,18,23,-1,1,-1,-1
14930746,1709396984692,4,16,650683235,1,-1,-1


In [20]:
submission = pd.read_csv('data/submission.csv', dtype='str', encoding='utf-8')
submission

,ego_id,u,v,x1
0,8,0,93,0.000000e+00
1,8,0,143,0.000000e+00
2,8,0,151,1.606742e+00
3,8,1,24,2.649638e-02
4,8,5,4,1.598572e-01
...,...,...,...,...
810971,1709396984676,73,23,1.562170e+00
810972,1709396984676,74,68,4.542098e-01
810973,1709396984676,77,28,7.850433e-02
810974,1709396984676,79,38,9.818121e-01


In [26]:
tezt = pd.merge(test, submission, how='inner', on=['ego_id', 'u', 'v'])
tezt

In [28]:
tezt = pd.merge(tezt, attr, how='left', on=['ego_id', 'u'])
tezt

,ego_id,u,v,t,x2,x3,x1,age,city_id,sex,school,university
0,8,7,16,422.7,0.0,0.0,2.277617e-02,37,979281502,1,213987831,562436811
1,8,8,29,NaN,0.0,0.0,8.581627e-02,121,-1,1,-1,-1
2,8,75,0,477.6,0.0,0.0,6.181129e-03,37,979281502,1,-1,845825535
3,8,0,151,0.2,1.3862943611198906,0.0,1.606742e+00,36,979281502,2,213987831,845825535
4,8,8,20,291.2,0.0,0.0,6.155013e-03,121,-1,1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
811028,1709396984676,79,38,50.2,0.0,0.0,9.818121e-01,16,269576388,2,566558003,-1
811029,1709396984676,77,28,43.4,0.0,0.0,7.850433e-02,16,926522633,1,43512792,112540362
811030,1709396984676,34,25,26.8,0.0,1.0,2.004123e-01,15,-1,1,253082810,-1
811031,1709396984676,0,21,6.6,0.0,0.0,1.621182e+00,14,-1,1,253082810,-1


In [30]:
tezt['ego_id'] = tezt['ego_id'].astype('str')
tezt['u'] = tezt['u'].astype('str')
tezt['v'] = tezt['v'].astype('str')
tezt['t'] = tezt['t'].astype('float').fillna(0.0)
tezt['city_id'] = tezt['city_id'].astype('str')
tezt['sex'] = tezt['sex'].astype('str')
tezt['school'] = tezt['school'].astype('str')
tezt['university'] = tezt['university'].astype('str')

In [31]:
X = ['x2', 'x3', 'age', 'city_id', 'sex', 'school', 'university']
cat_features = ['x3', 'city_id', 'sex', 'school', 'university']
# target (целевой признак)
y = ['x1']

# x2         - числовой
# x3         - категориальный
# age        - числовой
# city       - категориальный
# sex        - категориальный
# school     - категориальный
# university - категориальный

In [32]:
model.predict(tezt[X])

array([0.50173413, 0.57662617, 0.51359622, ..., 0.50631522, 0.83257438,
       0.69780924])

In [33]:
tezt['pred'] = model.predict(tezt[X])
tezt

,ego_id,u,v,t,x2,x3,x1,age,city_id,sex,school,university,pred
0,8,7,16,422.7,0.0,0.0,2.277617e-02,37,979281502,1,213987831,562436811,0.501734
1,8,8,29,0.0,0.0,0.0,8.581627e-02,121,-1,1,-1,-1,0.576626
2,8,75,0,477.6,0.0,0.0,6.181129e-03,37,979281502,1,-1,845825535,0.513596
3,8,0,151,0.2,1.3862943611198906,0.0,1.606742e+00,36,979281502,2,213987831,845825535,1.800465
4,8,8,20,291.2,0.0,0.0,6.155013e-03,121,-1,1,-1,-1,0.576626
...,...,...,...,...,...,...,...,...,...,...,...,...,...
811028,1709396984676,79,38,50.2,0.0,0.0,9.818121e-01,16,269576388,2,566558003,-1,0.563836
811029,1709396984676,77,28,43.4,0.0,0.0,7.850433e-02,16,926522633,1,43512792,112540362,0.712365
811030,1709396984676,34,25,26.8,0.0,1.0,2.004123e-01,15,-1,1,253082810,-1,0.506315
811031,1709396984676,0,21,6.6,0.0,0.0,1.621182e+00,14,-1,1,253082810,-1,0.832574


In [37]:
tezt['x1'] = tezt['pred']
tezt = tezt[['ego_id', 'u', 'v', 'x1']]
tezt = tezt.drop_duplicates(subset=['ego_id', 'u', 'v'])

,ego_id,u,v,x1
0,8,7,16,0.501734
1,8,8,29,0.576626
2,8,75,0,0.513596
3,8,0,151,1.800465
4,8,8,20,0.576626
...,...,...,...,...
811028,1709396984676,79,38,0.563836
811029,1709396984676,77,28,0.712365
811030,1709396984676,34,25,0.506315
811031,1709396984676,0,21,0.832574


In [40]:
tezt = tezt.drop_duplicates(subset=['ego_id', 'u', 'v'])
tezt.to_csv('prediction_1.csv', index=False, encoding='utf-8')

In [ ]:
# для submission нужно примерджить сюда данные по фичам из основного датасета и запустить предикт, посмотреть на сколько он совпадает

In [36]:
from sklearn.metrics import mean_squared_error


def calc_error(y_true, y_pred):
    print(mean_squared_error(y_true, y_pred, squared=True))


calc_error(
    tezt['x1'],
    tezt['pred']
)

1.2893625514217253


In [59]:
pd.read_csv('data/submission.csv', dtype='str', encoding='utf-8')

,ego_id,u,v,x1
0,8,0,93,0.000000e+00
1,8,0,143,0.000000e+00
2,8,0,151,1.606742e+00
3,8,1,24,2.649638e-02
4,8,5,4,1.598572e-01
...,...,...,...,...
810971,1709396984676,73,23,1.562170e+00
810972,1709396984676,74,68,4.542098e-01
810973,1709396984676,77,28,7.850433e-02
810974,1709396984676,79,38,9.818121e-01
